In [101]:
import nibabel as nb
import os
import sys
import numpy as np
from pprint import pprint

sys.path.append('../')

from neuroimg.base.objects.neuroimaging.baseneuroimage import RegionIndexMapping, ColorLut
from neuroimg.base.objects.neuroimaging.surface_object import GetSurface
from neuroimg.base.objects.neuroimaging.dataset import StructuralDataset
from neuroimg.base.objects.neuroimaging.baseneuroimage import pial_to_verts_and_triangs, read_cortical_region_mapping, \
    Hemisphere, RegionIndexMapping

SUBCORTICAL_REG_INDS = [8, 10, 11, 12, 13, 16, 17, 18, 26, 47, 49, 50, 51, 52, 53, 54, 58]
FS_LUT_LH_SHIFT = 1000
FS_LUT_RH_SHIFT = 2000

# Setup Data Directories

In [102]:
subjects_dir = "/home/adam2392/hdd/data/neuroimaging/freesurfer_output/outputfiles/la04/"
subjects_dir = "/home/adam2392/hdd/data/neuroimaging/freesurfer_output/la04/"
FREESURFER_HOME = os.getenv("FREESURFER_HOME") or ''
MRTRIX3_HOME = os.getenv("MRTRIX3_HOME") or ''
# define where lut files are
lut_fs=os.path.join(FREESURFER_HOME, "FreeSurferColorLUT.txt")
lut_target=os.path.join(MRTRIX3_HOME, "share/mrtrix3/labelconvert/fs_default.txt")

surfdir = os.path.join(subjects_dir, "surf")
subcortdir = os.path.join(subjects_dir, "aseg2srf")
labeldir = os.path.join(subjects_dir, "label")

source_lut = lut_fs
target_lut = lut_target

print("Subjects dir: ", subjects_dir)
print("Lut Fsurfer: ", lut_fs)
print("Lut MRTRIX: ", lut_target)

Subjects dir:  /home/adam2392/hdd/data/neuroimaging/freesurfer_output/la04/
Lut Fsurfer:  /opt/freesurfer/FreeSurferColorLUT.txt
Lut MRTRIX:  /opt/mrtrix3/share/mrtrix3/labelconvert/fs_default.txt


In [103]:
hemisphere = 'rh'
datafile = "/home/adam2392/Documents/eztrackv2/labels/rh_lobes.annot"

lobelabeldir = "/home/adam2392/Documents/eztrackv2/labels/"

# Read In Files

In [104]:
def get_cortical_regions(label_direc, region_index_mapping):
    # get the cortical region mapping for left/right hemispheres
    region_mapping_l = read_cortical_region_mapping(label_direc, Hemisphere.lh, region_index_mapping)
    region_mapping_r = read_cortical_region_mapping(label_direc, Hemisphere.rh, region_index_mapping)

    region_mappings = np.hstack([region_mapping_l, region_mapping_r])
    
    return region_mappings
    
def get_subcortical_regions(label_direc, subcort_surf_direc, region_index_mapping):
    allregionmaps = []
    
    # loop through all SUBCORTICAL REGION INDICES
    for fs_idx in SUBCORTICAL_REG_INDS:
        conn_idx = region_index_mapping.source_to_target(fs_idx)
        filename = os.path.join(subcort_surf_direc, 'aseg_%03d.srf' % fs_idx)
        with open(filename, 'r') as f:
            f.readline()
            nverts, ntriangs = [int(n) for n in f.readline().strip().split(' ')]

        vertices = np.genfromtxt(filename, dtype=float, skip_header=2, skip_footer=ntriangs, usecols=(0, 1, 2))
        triangles = np.genfromtxt(filename, dtype=int, skip_header=2 + nverts, usecols=(0, 1, 2))
            
        region_mapping = conn_idx * np.ones(nverts, dtype=int)
            
        allregionmaps.append(region_mapping)
    
    region_mappings = np.hstack(allregionmaps)
    return region_mappings
    
def read_cortical_lobe_mapping(datafile):
    # read in region mapping
    region_mapping, _, region_names = nb.freesurfer.io.read_annot(datafile)
    region_mapping = region_mapping - 1
    region_mapping[region_mapping == -2] = 0  # Unknown regions in hemispheres    
    return region_mapping, region_names

def get_cortical_lobes(label_direc):  
    lhfilename = os.path.join(label_direc, 'lh' + "_lobes.annot")
    rhfilename = os.path.join(label_direc, 'rh' + "_lobes.annot")
    
    # get the cortical region mapping for left/right hemispheres
    region_mapping_l, region_names_l = read_cortical_lobe_mapping(lhfilename)
    region_mapping_r, region_names_r = read_cortical_lobe_mapping(rhfilename)
    region_mappings = np.hstack([region_mapping_l, region_mapping_r])
    region_names = np.unique(np.hstack([region_names_l, region_names_r]))
    return region_mappings, region_names

def get_subcortical_lobes(label_direc):  
    allregionmaps = []
    
    # loop through all SUBCORTICAL REGION INDICES
    for fs_idx in SUBCORTICAL_REG_INDS:
        conn_idx = region_index_mapping.source_to_target(fs_idx)
        filename = os.path.join(subcort_surf_direc, 'aseg_%03d.srf' % fs_idx)
        with open(filename, 'r') as f:
            f.readline()
            nverts, ntriangs = [int(n) for n in f.readline().strip().split(' ')]

        vertices = np.genfromtxt(filename, dtype=float, skip_header=2, skip_footer=ntriangs, usecols=(0, 1, 2))
        triangles = np.genfromtxt(filename, dtype=int, skip_header=2 + nverts, usecols=(0, 1, 2))
            
        region_mapping = conn_idx * np.ones(nverts, dtype=int)
            
        allregionmaps.append(region_mapping)
        
        
    # get the cortical region mapping for left/right hemispheres
    region_mapping_l, region_names_l = read_cortical_lobe_mapping(lhfilename)
    region_mapping_r, region_names_r = read_cortical_lobe_mapping(rhfilename)
    region_mappings = np.hstack([region_mapping_l, region_mapping_r])
    region_names = np.unique(np.hstack([region_names_l, region_names_r]))
    return region_mappings, region_names

In [105]:
# create the region index mapping into the target lut space
region_index_mapping = RegionIndexMapping(source_lut, target_lut)

# compute the subcortical and cortical surfaces
subcort_regionmapping = get_subcortical_regions(labeldir, subcortdir,region_index_mapping)
cort_regionmapping = get_cortical_regions(labeldir,region_index_mapping)

In [106]:
print(subcort_regionmapping)
print(cort_regionmapping)

print(len(subcort_regionmapping), len(cort_regionmapping), len(subcort_regionmapping)+len(cort_regionmapping))
print(np.unique(subcort_regionmapping), np.unique(cort_regionmapping))

[35 35 35 ... 49 49 49]
[10 10 10 ... 60 60 60]
92824 321851 414675
[ 0 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 84] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 50 51 52 53 54 55 56 57 58 59 60 61 62
 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83]


In [107]:
# read lobe regions
lobe_mappings, lobe_names = get_cortical_lobes(lobelabeldir)

In [108]:
print(lobe_mappings)
print(lobe_names)

print(len(lobe_mappings))

[4 4 4 ... 0 0 0]
[b'cingulate' b'corpuscallosum' b'frontal' b'insula' b'occipital'
 b'parahippocampalgyrus' b'parietal' b'temporal' b'unknown']
321851


# Map Region Index to Lobe

In [88]:
# read in region mapping
region_mapping, _, region_names = nb.freesurfer.io.read_annot(origannotfile)
region_mapping = region_mapping - 1
region_mapping[region_mapping == -1] = 0  # Unknown regions in hemispheres    

print(np.unique(region_mapping))
print(region_mapping[:50])
print(len(region_mapping))

[-2  0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34]
[10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10]
162425


In [109]:
lobe_to_regind_dict = {}

# get unique region index on cortical
for regind in np.unique(cort_regionmapping):
    # get index where this region mapping ind occurs for cortical
    ind = np.where(cort_regionmapping == regind)[0]
    
    # get the lobe name corresponding to that index in the lobe mapping
    lobename = np.unique(lobe_names[lobe_mappings[ind]])
    
#     print(ind[0], regind, lobename)
    lobe_to_regind_dict[regind] = lobename

pprint(lobe_to_regind_dict)

{0: array([b'cingulate'], dtype='|S23'),
 1: array([b'parahippocampalgyrus'], dtype='|S23'),
 2: array([b'insula'], dtype='|S23'),
 3: array([b'cingulate'], dtype='|S23'),
 4: array([b'occipital'], dtype='|S23'),
 5: array([b'frontal'], dtype='|S23'),
 6: array([b'frontal'], dtype='|S23'),
 7: array([b'parietal'], dtype='|S23'),
 8: array([b'parahippocampalgyrus'], dtype='|S23'),
 9: array([b'insula'], dtype='|S23'),
 10: array([b'occipital'], dtype='|S23'),
 11: array([b'cingulate'], dtype='|S23'),
 12: array([b'occipital'], dtype='|S23'),
 13: array([b'cingulate'], dtype='|S23'),
 14: array([b'parahippocampalgyrus'], dtype='|S23'),
 15: array([b'frontal'], dtype='|S23'),
 16: array([b'cingulate'], dtype='|S23'),
 17: array([b'cingulate'], dtype='|S23'),
 18: array([b'cingulate'], dtype='|S23'),
 19: array([b'cingulate'], dtype='|S23'),
 20: array([b'occipital'], dtype='|S23'),
 21: array([b'parietal'], dtype='|S23'),
 22: array([b'insula'], dtype='|S23'),
 23: array([b'cingulate'], d

In [64]:
print(region_index_mapping.trg_table.names)

test =ColorLut(lut_fs)

['Unknown' 'ctx-lh-bankssts' 'ctx-lh-caudalanteriorcingulate'
 'ctx-lh-caudalmiddlefrontal' 'ctx-lh-cuneus' 'ctx-lh-entorhinal'
 'ctx-lh-fusiform' 'ctx-lh-inferiorparietal' 'ctx-lh-inferiortemporal'
 'ctx-lh-isthmuscingulate' 'ctx-lh-lateraloccipital'
 'ctx-lh-lateralorbitofrontal' 'ctx-lh-lingual'
 'ctx-lh-medialorbitofrontal' 'ctx-lh-middletemporal'
 'ctx-lh-parahippocampal' 'ctx-lh-paracentral' 'ctx-lh-parsopercularis'
 'ctx-lh-parsorbitalis' 'ctx-lh-parstriangularis' 'ctx-lh-pericalcarine'
 'ctx-lh-postcentral' 'ctx-lh-posteriorcingulate' 'ctx-lh-precentral'
 'ctx-lh-precuneus' 'ctx-lh-rostralanteriorcingulate'
 'ctx-lh-rostralmiddlefrontal' 'ctx-lh-superiorfrontal'
 'ctx-lh-superiorparietal' 'ctx-lh-superiortemporal'
 'ctx-lh-supramarginal' 'ctx-lh-frontalpole' 'ctx-lh-temporalpole'
 'ctx-lh-transversetemporal' 'ctx-lh-insula' 'Left-Cerebellum-Cortex'
 'Left-Thalamus-Proper' 'Left-Caudate' 'Left-Putamen' 'Left-Pallidum'
 'Left-Hippocampus' 'Left-Amygdala' 'Left-Accumbens-area'
 'R

../neuroimg/base/objects/neuroimaging/baseneuroimage.py:107: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  table = np.genfromtxt(os.fspath(filename), encoding="bytes", dtype=None)


In [67]:
print(ColorLut(lut_target).names)
print(test.names)

['Unknown' 'ctx-lh-bankssts' 'ctx-lh-caudalanteriorcingulate'
 'ctx-lh-caudalmiddlefrontal' 'ctx-lh-cuneus' 'ctx-lh-entorhinal'
 'ctx-lh-fusiform' 'ctx-lh-inferiorparietal' 'ctx-lh-inferiortemporal'
 'ctx-lh-isthmuscingulate' 'ctx-lh-lateraloccipital'
 'ctx-lh-lateralorbitofrontal' 'ctx-lh-lingual'
 'ctx-lh-medialorbitofrontal' 'ctx-lh-middletemporal'
 'ctx-lh-parahippocampal' 'ctx-lh-paracentral' 'ctx-lh-parsopercularis'
 'ctx-lh-parsorbitalis' 'ctx-lh-parstriangularis' 'ctx-lh-pericalcarine'
 'ctx-lh-postcentral' 'ctx-lh-posteriorcingulate' 'ctx-lh-precentral'
 'ctx-lh-precuneus' 'ctx-lh-rostralanteriorcingulate'
 'ctx-lh-rostralmiddlefrontal' 'ctx-lh-superiorfrontal'
 'ctx-lh-superiorparietal' 'ctx-lh-superiortemporal'
 'ctx-lh-supramarginal' 'ctx-lh-frontalpole' 'ctx-lh-temporalpole'
 'ctx-lh-transversetemporal' 'ctx-lh-insula' 'Left-Cerebellum-Cortex'
 'Left-Thalamus-Proper' 'Left-Caudate' 'Left-Putamen' 'Left-Pallidum'
 'Left-Hippocampus' 'Left-Amygdala' 'Left-Accumbens-area'
 'R

In [37]:
class ColorLut:
    """
    Class wrapper for the color lookup table.

    each column represents:
    id, name, R, G, B, A, shortname
    """

    def __init__(self, filename: os.PathLike):
        table = np.genfromtxt(os.fspath(filename), dtype=None)

        if len(table.dtype) == 6:
            # id name R G B A
            self.inds = table[table.dtype.names[0]]
            self.names = table[table.dtype.names[1]].astype('U')
            self.r = table[table.dtype.names[2]]
            self.g = table[table.dtype.names[3]]
            self.b = table[table.dtype.names[4]]
            self.a = table[table.dtype.names[5]]
            self.shortnames = np.zeros(len(self.names), dtype='U')

        elif len(table.dtype) == 7:
            # id shortname name R G B A
            self.inds = table[table.dtype.names[0]]
            self.shortnames = table[table.dtype.names[1]].astype('U')
            self.names = table[table.dtype.names[2]].astype('U')
            self.r = table[table.dtype.names[3]]
            self.g = table[table.dtype.names[4]]
            self.b = table[table.dtype.names[5]]
            self.a = table[table.dtype.names[6]]

            
class RegionIndexMapping:
    """
    Class wrapper for a region index mapping.

    This maps each index of the source file to an index in the corresponding look up table.
    """

    def __init__(self, color_lut_src_file: os.PathLike, color_lut_trg_file: os.PathLike):
        """
        :param color_lut_src_file: The source lookup table
        :param color_lut_trg_file: The target's lookup table.
        """
        self.src_table = ColorLut(color_lut_src_file)
        self.trg_table = ColorLut(color_lut_trg_file)

        names_to_trg = dict(zip(self.trg_table.names, self.trg_table.inds))

        self.src_to_trg = dict()
        for src_ind, src_name in zip(self.src_table.inds, self.src_table.names):
            trg_ind = names_to_trg.get(src_name, None)
            if trg_ind is not None:
                self.src_to_trg[src_ind] = trg_ind

        self.unknown_ind = names_to_trg.get('Unknown', 0)  # zero as the default unknown area

    def source_to_target(self, index):
        return self.src_to_trg.get(index, self.unknown_ind)



In [80]:
hemisphere = 'rh'
datafile = "/home/adam2392/Documents/eztrackv2/labels/rh_lobes.annot"
origannotfile = "/home/adam2392/hdd/data/neuroimaging/freesurfer_output/outputfiles/la04/lh.aparc.annot"

In [50]:
test = nb.freesurfer.io.read_annot(datafile)

print(test)
print(len(test))

(array([5, 5, 5, ..., 1, 1, 1], dtype=int32), array([[      25,        5,       25,        0,  1639705],
       [     100,       25,        0,        0,     6500],
       [     120,       70,       50,        0,  3294840],
       [     220,       20,       10,        0,   660700],
       [     140,       20,      140,        0,  9180300],
       [     120,      100,       60,        0,  3957880],
       [     140,      220,      220,        0, 14474380],
       [      80,      160,       20,        0,  1351760],
       [     255,      192,       32,        0,  2146559]], dtype=int32), [b'unknown', b'frontal', b'corpuscallosum', b'parahippocampalgyrus', b'cingulate', b'occipital', b'temporal', b'parietal', b'insula'])
3


In [53]:
test, colortable, region_names = nb.freesurfer.io.read_annot(origannotfile)

print(len(test))
print(len(region_names))
print(np.unique(test))
print(region_names)
print(test)
print(colortable.shape)

159426
36
[-1  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35]
[b'unknown', b'bankssts', b'caudalanteriorcingulate', b'caudalmiddlefrontal', b'corpuscallosum', b'cuneus', b'entorhinal', b'fusiform', b'inferiorparietal', b'inferiortemporal', b'isthmuscingulate', b'lateraloccipital', b'lateralorbitofrontal', b'lingual', b'medialorbitofrontal', b'middletemporal', b'parahippocampal', b'paracentral', b'parsopercularis', b'parsorbitalis', b'parstriangularis', b'pericalcarine', b'postcentral', b'posteriorcingulate', b'precentral', b'precuneus', b'rostralanteriorcingulate', b'rostralmiddlefrontal', b'superiorfrontal', b'superiorparietal', b'superiortemporal', b'supramarginal', b'frontalpole', b'temporalpole', b'transversetemporal', b'insula']
[11 11 11 ... 12 12 12]
(36, 5)


In [44]:

regionindexmapping = RegionIndexMapping(lut_fs, lut_target)
region_mapping, region_names = read_cortical_lobe_mapping(datafile, hemisphere, regionindexmapping)

print(len(region_mapping))
print(np.unique(region_mapping))
print(region_names)
print(region_mapping)

159426
[ 0 50 52 53 54 55 56]
[b'unknown', b'frontal', b'corpuscallosum', b'parahippocampalgyrus', b'cingulate', b'occipital', b'temporal', b'parietal', b'insula']
[53 53 53 ... 50 50 50]


/home/adam2392/anaconda3/envs/neuroimgpipe/lib/python3.6/site-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  # Remove the CWD from sys.path while we load stuff.


In [41]:
def read_cortical_region_mapping(label_direc: os.PathLike, 
                                 hemisphere: Hemisphere, 
                                 fs_to_conn: RegionIndexMapping) \
        -> np.ndarray:
    """
    Reads the cortical region mapping file.

    :param label_direc: Where the annotation label directory is
    :param hemisphere: (Hemisphere) enumerator
    :param fs_to_conn: (RegionIndexMapping)
    :return:
    """
    filename = os.path.join(label_direc, hemisphere.value + ".aparc.annot")
    region_mapping, _, _ = nibabel.freesurfer.io.read_annot(filename)

    region_mapping[region_mapping == -1] = 0  # Unknown regions in hemispheres

    # $FREESURFER_HOME/FreeSurferColorLUT.txt describes the shift
    if hemisphere == Hemisphere.lh:
        region_mapping += FS_LUT_LH_SHIFT
    else:
        region_mapping += FS_LUT_RH_SHIFT

    fs_to_conn_fun = np.vectorize(lambda n: fs_to_conn.source_to_target(n))
    region_mapping = fs_to_conn_fun(region_mapping)

    return region_mapping


NameError: name 'Hemisphere' is not defined